In [1]:
import pandas as pd
import numpy as np
import sys

In [2]:
faults = pd.read_csv('../data/J1939Faults.csv', low_memory=False)

In [3]:
faults.head(3)

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000


In [4]:
#count of trucks
faults.groupby('EquipmentID').count()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,MCTNumber,Latitude,Longitude,LocationTimeStamp
EquipmentID,,,,,,,,,,,,,,,,,,,
0105309016,1,1,1,1,0,1,1,1,1,1,1,1,1,1,0,1,1,1,1
0105319027,9,9,9,9,0,9,9,9,9,9,9,9,9,9,0,9,9,9,9
0105355995,1,1,1,1,0,1,1,1,1,1,1,1,1,1,0,1,1,1,1
0105406655,6,6,6,6,0,3,2,5,5,6,6,6,6,6,0,6,6,6,6
0105438416,2,2,2,1,0,2,2,2,2,2,2,2,2,2,0,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,496,496,496,468,0,471,467,485,485,496,496,496,496,496,0,496,496,496,496
309,5991,5991,5991,5977,0,5538,5512,5740,5740,5991,5991,5991,5991,5991,0,5991,5991,5991,5991
310,1184,1184,1184,1156,0,1116,1108,1157,1157,1184,1184,1184,1184,1184,0,1184,1184,1184,1184


In [5]:
faults.shape

(1187335, 20)

In [6]:
for col in ['EventTimeStamp', 'LocationTimeStamp']:
    faults[col]=faults[col].astype('datetime64[ns]')

In [7]:
faults.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187335 entries, 0 to 1187334
Data columns (total 20 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   RecordID               1187335 non-null  int64         
 1   ESS_Id                 1187335 non-null  int64         
 2   EventTimeStamp         1187335 non-null  datetime64[ns]
 3   eventDescription       1126490 non-null  object        
 4   actionDescription      0 non-null        float64       
 5   ecuSoftwareVersion     891285 non-null   object        
 6   ecuSerialNumber        844318 non-null   object        
 7   ecuModel               1122577 non-null  object        
 8   ecuMake                1122577 non-null  object        
 9   ecuSource              1187335 non-null  int64         
 10  spn                    1187335 non-null  int64         
 11  fmi                    1187335 non-null  int64         
 12  active                 11873

#### Remove faults occurring in the vicinity of the service locations at (36.0666667, -86.4347222), (35.5883333, -86.4438888), and (36.1950, -83.174722)

In [8]:
service_loc1=faults[(faults['Latitude']==36.066666)&(faults['Longitude']== -86.434722)]
service_loc2=faults[(faults['Latitude']==35.588333)& (faults['Longitude']==-86.443888)]
service_loc3=faults[(faults['Latitude']==36.1950) & (faults['Longitude']==-83.174722)]

service_locs=pd.concat([service_loc1, service_loc2, service_loc3])
faults_cleaned=faults.drop(service_locs.index)

In [9]:
service_locs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 477 entries, 479 to 1127328
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   RecordID               477 non-null    int64         
 1   ESS_Id                 477 non-null    int64         
 2   EventTimeStamp         477 non-null    datetime64[ns]
 3   eventDescription       441 non-null    object        
 4   actionDescription      0 non-null      float64       
 5   ecuSoftwareVersion     377 non-null    object        
 6   ecuSerialNumber        355 non-null    object        
 7   ecuModel               449 non-null    object        
 8   ecuMake                449 non-null    object        
 9   ecuSource              477 non-null    int64         
 10  spn                    477 non-null    int64         
 11  fmi                    477 non-null    int64         
 12  active                 477 non-null    bool          
 13 

In [10]:
faults_cleaned.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25
1,2,990360,2015-02-21 11:34:34,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10
2,3,990364,2015-02-21 11:35:31,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26
3,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08
4,5,990416,2015-02-21 11:39:41,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37


#### Remove Equipment ID more than 5 character 

In [11]:
faults_cleaned['EquipmentID'].sort_values()

1008925    0105309016
985183     0105319027
985277     0105319027
985321     0105319027
985237     0105319027
              ...    
4245            R1762
4494            R1762
4427            R1762
4999            R1764
4952            R1764
Name: EquipmentID, Length: 1186858, dtype: object

In [12]:
faults_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1186858 entries, 0 to 1187334
Data columns (total 20 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   RecordID               1186858 non-null  int64         
 1   ESS_Id                 1186858 non-null  int64         
 2   EventTimeStamp         1186858 non-null  datetime64[ns]
 3   eventDescription       1126049 non-null  object        
 4   actionDescription      0 non-null        float64       
 5   ecuSoftwareVersion     890908 non-null   object        
 6   ecuSerialNumber        843963 non-null   object        
 7   ecuModel               1122128 non-null  object        
 8   ecuMake                1122128 non-null  object        
 9   ecuSource              1186858 non-null  int64         
 10  spn                    1186858 non-null  int64         
 11  fmi                    1186858 non-null  int64         
 12  active                 11868

In [13]:
faults_cleaned=faults_cleaned[faults_cleaned['EquipmentID'].map(len)<=5]

In [14]:
#want to see everything in cell 15
np.set_printoptions(threshold=sys.maxsize)

In [15]:
faults_cleaned['EquipmentID'].unique()

array(['1439', '1369', '1674', '1417', '1597', '1429', '1582', '309',
       '1601', '302', '310', '1733', '1590', '1605', '1644', '1585',
       '1634', '1442', '1404', '1620', '1721', '1610', '1576', '1395',
       '1766', '1437', '305', '1586', '1600', '1558', '1630', '1646',
       '1595', '1432', '1377', '1641', '1469', '1371', '1365', '1677',
       '1609', '1612', '1768', '1650', '1657', '1514', '1703', '1451',
       '1625', '1565', '1520', '1623', '1363', '1569', '1390', '1607',
       '1776', '1370', '1474', '1673', '1749', '1658', '1449', '1566',
       '1499', '1427', '1688', '1467', '1669', '1606', '1462', '1553',
       '1683', '1391', '1458', '1727', '1632', '1778', '1559', '1431',
       '1372', '1769', '1492', '1434', '1611', '1402', '1460', '1481',
       '1555', '1375', '1651', '1419', '1580', '1374', '301', '1401',
       '1614', '1737', '1358', '1740', '1562', '1339', '1715', '1405',
       '1654', '1594', '1729', '1571', '1337', '1560', '1455', '1649',
       '144

In [16]:
top_10_desc=faults_cleaned['eventDescription'].value_counts().head(10)
top_10_desc

Low (Severity Low) Engine Coolant Level                                                      320151
Abnormal Update Rate Tire Location                                                           256458
High Voltage (Fuel Level)                                                                     82786
High Voltage (Left Fuel Level Sensor)                                                         81732
Low (Severity Medium) Engine Coolant Level                                                    38503
Incorrect Data J1939 Network #1 Primary Vehicle Network ( previously SAE J1939 Data Link)     36918
Condition Exists Cruise Control Enable Switch                                                 22554
High (Severity Low) Water In Fuel Indicator                                                   17674
Condition Exists Engine Protection Torque Derate                                              10711
High Voltage (Water In Fuel Indicator)                                                         8627


In [17]:
faults_cleaned

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25
1,2,990360,2015-02-21 11:34:34,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10
2,3,990364,2015-02-21 11:35:31,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26
3,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08
4,5,990416,2015-02-21 11:39:41,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187330,1248454,123904424,2020-03-06 14:00:26,Low (Severity Low) Catalyst Tank Level,NaN,04384413*22383729*082218154102*60701732*G1*BGT*,80156139,6X1u17D1500000000,CMMNS,0,1761,17,False,3,NaN,2282,105439740,37.094768,-85.897407,2020-03-06 14:00:21
1187331,1248455,123905139,2020-03-06 14:04:23,Condition Exists Engine Protection Torque Derate,NaN,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,1569,31,True,5,NaN,1994,105354084,34.390740,-79.461805,2020-03-06 14:04:59
1187332,1248456,123905996,2020-03-06 14:13:38,Abnormal Rate of Change Aftertreatment 1 Intak...,NaN,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,3216,10,True,1,NaN,1850,105336308,34.430370,-84.920509,2020-03-06 14:14:14
1187333,1248457,123906113,2020-03-06 14:14:13,Low (Severity Medium) Engine Coolant Level,NaN,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,18,True,8,NaN,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49


In [18]:
spn_5264=faults_cleaned[faults_cleaned['spn']== 5246] #spn=5246 means that the truck is towed 
spn_5264

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
45,46,990931,2015-02-21 12:10:51,NaN,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,5246,0,True,1,NaN,1395,105349612,36.065972,-86.433425,2015-02-21 12:11:27
1918,1919,1007751,2015-02-22 19:44:55,NaN,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,5246,0,True,1,NaN,1395,105349612,36.066203,-86.434814,2015-02-22 19:46:27
2058,2059,1010486,2015-02-23 04:00:21,NaN,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,5246,0,False,1,NaN,1395,105349612,36.066666,-86.434537,2015-02-23 01:06:06
2089,2090,1011009,2015-02-23 05:05:44,NaN,NaN,05290170*03015749*051914190353*09400015*G1*BDR*,79642446,6X1u13D1500000000,CMMNS,0,5246,0,True,1,NaN,1630,105329900,40.733009,-74.087777,2015-02-23 05:08:23
2971,2972,1026305,2015-02-23 15:54:22,NaN,NaN,unknown,unknown,unknown,unknown,0,5246,0,True,1,NaN,1487,105369355,28.077361,-81.897083,2015-02-23 15:54:58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183032,1244156,121610128,2020-02-19 07:02:33,NaN,NaN,05317106*05005224*051718172255*09401583*G1*BDR*,79845785,6X1u13D1500000000,CMMNS,0,5246,0,True,1,NaN,1814,105369518,36.067037,-86.434120,2020-02-19 07:03:09
1183684,1244808,121909497,2020-02-21 07:23:44,NaN,NaN,04384413*22246857*090619141107*60701756*G1*BGT*,80092582,6X1u17D1500000000,CMMNS,0,5246,16,True,1,NaN,2211,105329862,36.066296,-86.434305,2020-02-21 07:24:20
1184328,1245452,122305094,2020-02-24 15:28:05,NaN,NaN,04384413*22246857*090619141107*60701756*G1*BGT*,80092582,6X1u17D1500000000,CMMNS,0,5246,16,False,1,NaN,2211,105329862,36.066620,-86.434722,2020-02-24 15:28:01
1184330,1245454,122305096,2020-02-24 15:27:26,NaN,NaN,04384413*22246857*090619141107*60701756*G1*BGT*,80092582,6X1u17D1500000000,CMMNS,0,5246,0,True,1,NaN,2211,105329862,36.066620,-86.434722,2020-02-24 15:28:02


In [19]:
diagnostics=pd.read_csv('../data/VehicleDiagnosticOnboardData.csv')

In [20]:
diagnostics.head()

,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1


In [21]:
faults_cleaned[~faults_cleaned['EquipmentID'].isin(faults_cleaned.loc[faults_cleaned['spn'] == 5246, 'EquipmentID'].unique())]


,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25
1,2,990360,2015-02-21 11:34:34,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10
2,3,990364,2015-02-21 11:35:31,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26
3,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08
4,5,990416,2015-02-21 11:39:41,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187330,1248454,123904424,2020-03-06 14:00:26,Low (Severity Low) Catalyst Tank Level,NaN,04384413*22383729*082218154102*60701732*G1*BGT*,80156139,6X1u17D1500000000,CMMNS,0,1761,17,False,3,NaN,2282,105439740,37.094768,-85.897407,2020-03-06 14:00:21
1187331,1248455,123905139,2020-03-06 14:04:23,Condition Exists Engine Protection Torque Derate,NaN,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,1569,31,True,5,NaN,1994,105354084,34.390740,-79.461805,2020-03-06 14:04:59
1187332,1248456,123905996,2020-03-06 14:13:38,Abnormal Rate of Change Aftertreatment 1 Intak...,NaN,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,3216,10,True,1,NaN,1850,105336308,34.430370,-84.920509,2020-03-06 14:14:14
1187333,1248457,123906113,2020-03-06 14:14:13,Low (Severity Medium) Engine Coolant Level,NaN,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,18,True,8,NaN,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49


In [22]:
diagnostics.shape

(12821626, 4)

In [23]:
diagnostics.loc[diagnostics['Value'].str.contains(',')]

,Id,Name,Value,FaultId
200754,201459,EngineOilPressure,"75,98",18402
200755,201460,EngineOilTemperature,"177,575",18402
200757,201462,TurboBoostPressure,"33,35",18402
200760,201465,FuelLtd,"17867,93716715",18402
200761,201466,FuelRate,"20,68474",18402
...,...,...,...,...
5632164,5640222,IntakeManifoldTemperature,"98,6",545727
5632166,5640224,EngineRpm,"1261,5",545727
5632167,5640225,BarometricPressure,"14,2825",545727
5632169,5640227,Speed,"64,47697",545727


In [24]:
#another approch for slicing the data
diagnostics[diagnostics['Value'].str.contains(',')]

,Id,Name,Value,FaultId
200754,201459,EngineOilPressure,"75,98",18402
200755,201460,EngineOilTemperature,"177,575",18402
200757,201462,TurboBoostPressure,"33,35",18402
200760,201465,FuelLtd,"17867,93716715",18402
200761,201466,FuelRate,"20,68474",18402
...,...,...,...,...
5632164,5640222,IntakeManifoldTemperature,"98,6",545727
5632166,5640224,EngineRpm,"1261,5",545727
5632167,5640225,BarometricPressure,"14,2825",545727
5632169,5640227,Speed,"64,47697",545727


In [25]:
#to convert 'string' to 'float', need to change , to .
diagnostics['Value']=diagnostics['Value'].str.replace(',', '.')

In [26]:
#pivoting diagnostics
diagnostics=diagnostics.pivot(index=['FaultId'], columns='Name', values='Value')
diagnostics

Name,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
FaultId,,,,,,,,,,,,,,,,,,,,,
1,0,14.21,False,66.48672,423178.7,100.4,11,0,96.74375,0,...,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1248454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
1248455,100,14.5,True,64.6226,423937.9,185,51,37.12,211.4937,1310.25,...,32,True,98.6,18431,False,NaN,65.01096,NaN,73.2,7.83
1248456,0,14.355,True,66.48672,465925.4,186.8,62,41.18,212.8438,1340.75,...,NaN,True,91.4,17407,NaN,NaN,66.5741,NaN,100,6.96


In [27]:
diagnostics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1187335 entries, 1 to 1248458
Data columns (total 24 columns):
 #   Column                     Non-Null Count    Dtype 
---  ------                     --------------    ----- 
 0   AcceleratorPedal           531889 non-null   object
 1   BarometricPressure         585976 non-null   object
 2   CruiseControlActive        574916 non-null   object
 3   CruiseControlSetSpeed      576458 non-null   object
 4   DistanceLtd                585819 non-null   object
 5   EngineCoolantTemperature   586071 non-null   object
 6   EngineLoad                 585621 non-null   object
 7   EngineOilPressure          586244 non-null   object
 8   EngineOilTemperature       583912 non-null   object
 9   EngineRpm                  586921 non-null   object
 10  EngineTimeLtd              581366 non-null   object
 11  FuelLevel                  502795 non-null   object
 12  FuelLtd                    585195 non-null   object
 13  FuelRate                   

In [28]:
diagnostics.columns

Index(['AcceleratorPedal', 'BarometricPressure', 'CruiseControlActive',
       'CruiseControlSetSpeed', 'DistanceLtd', 'EngineCoolantTemperature',
       'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm',
       'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 'FuelRate', 'FuelTemperature',
       'IgnStatus', 'IntakeManifoldTemperature', 'LampStatus', 'ParkingBrake',
       'ServiceDistance', 'Speed', 'SwitchedBatteryVoltage', 'Throttle',
       'TurboBoostPressure'],
      dtype='object', name='Name')

In [29]:
float_columns=['AcceleratorPedal', 'BarometricPressure',
       'CruiseControlSetSpeed', 'DistanceLtd', 'EngineCoolantTemperature',
       'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm',
       'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 'FuelRate', 'FuelTemperature',
        'IntakeManifoldTemperature', 'LampStatus', 
       'ServiceDistance', 'Speed', 'SwitchedBatteryVoltage', 'Throttle',
       'TurboBoostPressure']

for col in float_columns:
    diagnostics[col]=diagnostics[col].astype(float)

In [30]:
bool_columns=['CruiseControlActive', 'IgnStatus', 'ParkingBrake']
#boolean_mapper={'True':True, 'False':False}

for col in bool_columns:
    diagnostics[col]=diagnostics[col].map({'True': True, 'False': False})

In [31]:
diagnostics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1187335 entries, 1 to 1248458
Data columns (total 24 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   AcceleratorPedal           531889 non-null   float64
 1   BarometricPressure         585976 non-null   float64
 2   CruiseControlActive        574916 non-null   object 
 3   CruiseControlSetSpeed      576458 non-null   float64
 4   DistanceLtd                585819 non-null   float64
 5   EngineCoolantTemperature   586071 non-null   float64
 6   EngineLoad                 585621 non-null   float64
 7   EngineOilPressure          586244 non-null   float64
 8   EngineOilTemperature       583912 non-null   float64
 9   EngineRpm                  586921 non-null   float64
 10  EngineTimeLtd              581366 non-null   float64
 11  FuelLevel                  502795 non-null   float64
 12  FuelLtd                    585195 non-null   float64
 13  FuelRate    

In [32]:
#merging faults with diagnostics
faults_diagnostics = faults_cleaned.merge(diagnostics, left_on = 'RecordID', right_on = 'FaultId') \
    .set_index('EventTimeStamp').sort_index()


In [33]:
faults_diagnostics.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1184701 entries, 2000-03-18 19:14:10 to 2026-05-16 14:44:11
Data columns (total 43 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   RecordID                   1184701 non-null  int64         
 1   ESS_Id                     1184701 non-null  int64         
 2   eventDescription           1124368 non-null  object        
 3   actionDescription          0 non-null        float64       
 4   ecuSoftwareVersion         888974 non-null   object        
 5   ecuSerialNumber            842051 non-null   object        
 6   ecuModel                   1120080 non-null  object        
 7   ecuMake                    1120080 non-null  object        
 8   ecuSource                  1184701 non-null  int64         
 9   spn                        1184701 non-null  int64         
 10  fmi                        1184701 non-null  int64         
 11  acti

In [34]:
#spn=1569 means that truck is almost (75%) towed 
faults_cleaned[faults_cleaned['spn']==1569]

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
40,41,990856,2015-02-21 12:06:22,Condition Exists Engine Protection Torque Derate,NaN,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,True,5,NaN,1721,105439606,39.051805,-84.560509,2015-02-21 12:06:57
249,250,993542,2015-02-21 15:01:49,Condition Exists Engine Protection Torque Derate,NaN,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,False,5,NaN,1721,105439606,38.113240,-85.667361,2015-02-21 15:01:45
290,291,994045,2015-02-21 15:35:45,Condition Exists Engine Protection Torque Derate,NaN,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,True,6,NaN,1721,105439606,37.735185,-85.808101,2015-02-21 15:36:21
306,307,994237,2015-02-21 15:51:58,Condition Exists Engine Protection Torque Derate,NaN,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,False,6,NaN,1721,105439606,37.540370,-85.880972,2015-02-21 15:51:54
340,341,994680,2015-02-21 16:22:24,Condition Exists Engine Protection Torque Derate,NaN,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,True,7,NaN,1721,105439606,37.166666,-85.964027,2015-02-21 16:23:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1186072,1247196,123246301,2020-03-02 15:51:18,Condition Exists Engine Protection Torque Derate,NaN,04358814*06009282*100515184137*09400147*G1*BDR*,79897319,6X1u13D1500000000,CMMNS,0,1569,31,True,10,NaN,1855,105317775,33.757453,-84.548611,2020-03-02 15:43:54
1186195,1247319,123306956,2020-03-03 06:51:50,Condition Exists Engine Protection Torque Derate,NaN,05317106*05088449*051718172255*09401583*G1*BDR*,79875897,6X1u13D1500000000,CMMNS,0,1569,31,False,1,NaN,1827,105464811,36.194907,-83.174953,2020-03-03 06:51:46
1186207,1247331,123307779,2020-03-03 06:59:13,Condition Exists Engine Protection Torque Derate,NaN,05317106*05088449*051718172255*09401583*G1*BDR*,79875897,6X1u13D1500000000,CMMNS,0,1569,31,False,1,NaN,1827,105464811,36.194861,-83.175000,2020-03-03 07:00:53
1186730,1247854,123594250,2020-03-04 16:55:48,Condition Exists Engine Protection Torque Derate,NaN,04358814*06027162*051718174436*09401683*G1*BDR*,79902732,6X1u13D1500000000,CMMNS,0,1569,31,False,1,NaN,1868,105420782,35.587870,-86.443935,2020-03-04 16:55:44


In [35]:
#derates = faults_cleaned[((faults_cleaned['spn'] == 1569) & (faults_cleaned['fmi'] == 31))| (faults_cleaned['spn'] == 5246)& (faults_cleaned['fmi'] == 30)]


In [36]:
#creating a dataframe containing only derated trucks
derates = faults_cleaned[((faults_cleaned['spn'] == 1569) & (faults_cleaned['fmi'] == 31))| (faults_cleaned['spn'] == 5246)& (faults_cleaned['fmi'] == 0)]
derates

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
40,41,990856,2015-02-21 12:06:22,Condition Exists Engine Protection Torque Derate,NaN,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,True,5,NaN,1721,105439606,39.051805,-84.560509,2015-02-21 12:06:57
45,46,990931,2015-02-21 12:10:51,NaN,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,5246,0,True,1,NaN,1395,105349612,36.065972,-86.433425,2015-02-21 12:11:27
249,250,993542,2015-02-21 15:01:49,Condition Exists Engine Protection Torque Derate,NaN,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,False,5,NaN,1721,105439606,38.113240,-85.667361,2015-02-21 15:01:45
290,291,994045,2015-02-21 15:35:45,Condition Exists Engine Protection Torque Derate,NaN,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,True,6,NaN,1721,105439606,37.735185,-85.808101,2015-02-21 15:36:21
306,307,994237,2015-02-21 15:51:58,Condition Exists Engine Protection Torque Derate,NaN,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,False,6,NaN,1721,105439606,37.540370,-85.880972,2015-02-21 15:51:54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1186072,1247196,123246301,2020-03-02 15:51:18,Condition Exists Engine Protection Torque Derate,NaN,04358814*06009282*100515184137*09400147*G1*BDR*,79897319,6X1u13D1500000000,CMMNS,0,1569,31,True,10,NaN,1855,105317775,33.757453,-84.548611,2020-03-02 15:43:54
1186195,1247319,123306956,2020-03-03 06:51:50,Condition Exists Engine Protection Torque Derate,NaN,05317106*05088449*051718172255*09401583*G1*BDR*,79875897,6X1u13D1500000000,CMMNS,0,1569,31,False,1,NaN,1827,105464811,36.194907,-83.174953,2020-03-03 06:51:46
1186207,1247331,123307779,2020-03-03 06:59:13,Condition Exists Engine Protection Torque Derate,NaN,05317106*05088449*051718172255*09401583*G1*BDR*,79875897,6X1u13D1500000000,CMMNS,0,1569,31,False,1,NaN,1827,105464811,36.194861,-83.175000,2020-03-03 07:00:53
1186730,1247854,123594250,2020-03-04 16:55:48,Condition Exists Engine Protection Torque Derate,NaN,04358814*06027162*051718174436*09401683*G1*BDR*,79902732,6X1u13D1500000000,CMMNS,0,1569,31,False,1,NaN,1868,105420782,35.587870,-86.443935,2020-03-04 16:55:44


In [37]:
#size of the derated trucks
derates.groupby(['spn','fmi'])['eventDescription'].size()


spn   fmi
1569  31     10711
5246  0        784
Name: eventDescription, dtype: int64

In [38]:
#unique equipment ID in derated trucks
derates['EquipmentID'].unique()

array(['1721', '1395', '1515', '1630', '1487', '1499', '1329', '1365',
       '1419', '1486', '1509', '1492', '1340', '1328', '1430', '1370',
       '1497', '1350', '1473', '1608', '1606', '1612', '1354', '1493',
       '1583', '1465', '1335', '1789', '1383', '1389', '1501', '1438',
       '1463', '1514', '1666', '1689', '1692', '1357', '1396', '310',
       '1602', '1660', '1445', '1431', '1327', '1795', '1399', '1494',
       '1367', '1364', '1452', '1595', '1740', '1366', '1339', '1425',
       '1358', '1646', '1391', '1428', '1338', '1637', '1346', '1490',
       '1562', '1433', '1663', '1417', '1424', '1771', '1790', '1591',
       '1797', '1596', '1421', '1444', '1480', '1466', '1457', '1794',
       '1696', '1703', '1626', '1715', '1437', '1453', '1584', '1597',
       '1610', '1377', '1652', '1517', '1784', '1751', '1786', '1590',
       '1341', '1382', '1714', '1711', '1369', '1398', '1791', '1772',
       '1472', '1418', '1645', '1475', '1815', '1806', '1607', '1560',
       

In [39]:
#create a list of trucks that reach a derate. this may make the EquipmentID <= 5 irrelevant since none of these
#are longer than 4 characters.

derates_list = ['1721', '1395', '1515', '1630', '1487', '1499', '1329', '1365',
       '1419', '1486', '1509', '1492', '1340', '1328', '1430', '1370',
       '1497', '1350', '1473', '1608', '1606', '1612', '1354', '1493',
       '1583', '1465', '1335', '1789', '1383', '1389', '1501', '1438',
       '1463', '1514', '1666', '1689', '1692', '1357', '1396', '310',
       '1602', '1660', '1445', '1431', '1327', '1795', '1399', '1494',
       '1367', '1364', '1452', '1595', '1740', '1366', '1339', '1425',
       '1358', '1646', '1391', '1428', '1338', '1637', '1346', '1490',
       '1562', '1433', '1663', '1417', '1424', '1771', '1790', '1591',
       '1797', '1596', '1421', '1444', '1480', '1466', '1457', '1794',
       '1696', '1703', '1626', '1715', '1437', '1453', '1584', '1597',
       '1610', '1377', '1652', '1517', '1784', '1751', '1786', '1590',
       '1341', '1382', '1714', '1711', '1369', '1398', '1791', '1772',
       '1472', '1418', '1645', '1475', '1815', '1806', '1607', '1560',
       '1450', '1401', '1572', '1375', '1368', '1742', '1650', '1443',
       '1495', '1670', '1587', '1778', '1566', '1617', '1691', '1573',
       '1477', '1476', '1524', '1441', '1405', '1577', '1447', '1659',
       '1408', '1439', '1403', '1458', '1668', '1814', '1567', '1535',
       '1373', '1446', '1700', '1629', '1730', '1665', '1381', '1551',
       '1413', '1643', '1508', '1440', '1455', '1844', '1571', '1429',
       '1491', '1803', '1467', '1378', '1488', '1739', '1448', '1420',
       '1384', '1683', '1372', '1502', '1582', '1464', '1599', '1712',
       '1407', '1603', '1644', '1757', '1618', '1604', '1519', '1831',
       '1631', '1811', '1628', '1818', '1423', '1636', '1555', '1719',
       '1638', '1710', '1823', '1435', '1808', '1392', '1669', '1649',
       '1579', '1416', '1575', '1397', '1534', '1539', '1510', '1813',
       '1601', '1657', '1621', '1651', '1620', '1708', '1592', '1613',
       '1619', '1925', '1887', '1662', '1640', '1881', '1563', '1713',
       '1557', '1839', '1833', '1605', '1513', '1639', '1866', '1485',
       '1581', '1549', '1776', '308', '1586', '1661', '1635', '1907',
       '1853', '1785', '1594', '1859', '1600', '1963', '1848', '1918',
       '1561', '1796', '1624', '1598', '1972', '1706', '1641', '1874',
       '1940', '1609', '1642', '1506', '1552', '1685', '1883', '1946',
       '1913', '1920', '1634', '1995', '1512', '1878', '1564', '1686',
       '1554', '1505', '1802', '1788', '1868', '1807', '1688', '1922',
       '1903', '1681', '1904', '1615', '1828', '1694', '1559', '1827',
       '1842', '1720', '1970', '1877', '1888', '1989', '1932', '1804',
       '1854', '1525', '1521', '1851', '1565', '1981', '1741', '1980',
       '1737', '1580', '1547', '1924', '1822', '1766', '1812', '1917',
       '1933', '1817', '1654', '1731', '2011', '2017', '2008', '1726',
       '1717', '1656', '1835', '1664', '1880', '1916', '1996', '1912',
       '1947', '1997', '1736', '1820', '1729', '1864', '1528', '1556',
       '1898', '1884', '1832', '1705', '1585', '1908', '1752', '1623',
       '1800', '1732', '1725', '1845', '1876', '1576', '1897', '1543',
       '1647', '1931', '1971', '1852', '1697', '1959', '1953', '1787',
       '1962', '1893', '2015', '2019', '1957', '1593', '1965', '1873',
       '1911', '1934', '1942', '1982', '1900', '2009', '1948', '1723',
       '2006', '1956', '2012', '1836', '1935', '307', '1879', '1792',
       '1865', '1885', '1690', '1793', '1860', '1894', '1856', '1704',
       '1614', '1655', '1667', '1632', '1699', '1687', '1625', '1871',
       '1750', '1611', '2013', '1939', '1886', '1999', '1819', '1901',
       '1821', '1847', '1701', '1588', '1838', '1849', '1775', '1743',
       '1952', '1988', '1716', '1862', '1863', '1622', '2004', '1764',
       '1905', '1698', '1846', '309', '1798', '1780', '1762', '1979',
       '2007', '1919', '1966', '1843', '1938', '1744', '1992', '1976',
       '1926', '2143', '1718', '1899', '2111', '1978', '1763', '1870',
       '1889', '1994', '2027', '1967', '1928', '1921', '1891', '1734',
       '1945', '2010', '306', '1951', '1768', '1781', '2002', '1728',
       '1869', '1735', '1964', '1858', '1758', '1958', '2120', '1756',
       '2209', '1944', '1927', '2005', '1709', '2082', '1914', '2020',
       '1872', '1975', '1857', '1733', '1961', '1954', '1985', '1824',
       '1993', '1895', '1906', '2018', '1977', '1834', '1930', '1969',
       '1991', '1984', '1949', '1896', '2109', '2030', '1950', '1855',
       '2000', '1986', '1782', '1937', '2021', '1943', '2089', '1968',
       '1909', '1829', '1861', '2175', '2211']

faults_cleaned = faults_cleaned[faults_cleaned['EquipmentID'].isin(derates_list)]

In [40]:
#merge our cleaned faults_cleaned dataframe with diagnostics 
faults_diagnostics = faults_cleaned.merge(diagnostics, left_on = 'RecordID', right_on = 'FaultId') \
    .set_index('EventTimeStamp').sort_index()

In [41]:
faults_diagnostics.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 921737 entries, 2000-03-18 19:14:10 to 2026-05-16 14:44:11
Data columns (total 43 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   RecordID                   921737 non-null  int64         
 1   ESS_Id                     921737 non-null  int64         
 2   eventDescription           889644 non-null  object        
 3   actionDescription          0 non-null       float64       
 4   ecuSoftwareVersion         666527 non-null  object        
 5   ecuSerialNumber            623200 non-null  object        
 6   ecuModel                   867546 non-null  object        
 7   ecuMake                    867546 non-null  object        
 8   ecuSource                  921737 non-null  int64         
 9   spn                        921737 non-null  int64         
 10  fmi                        921737 non-null  int64         
 11  active            

In [42]:
#create new csv of the cleaned and merged dataset.
faults_diagnostics.to_csv('../data/faults_diagnostics.csv')

In [43]:
cumulative_active = pd.DataFrame(derates.groupby(['EquipmentID', 'EventTimeStamp','spn'])['active'].sum().groupby(level = 0).cumsum()).reset_index()
cumulative_active


,EquipmentID,EventTimeStamp,spn,active
0,1327,2015-05-28 09:45:18,1569,1
1,1327,2015-06-01 11:04:31,1569,1
2,1327,2015-06-01 11:05:36,1569,2
3,1327,2015-06-01 11:08:19,1569,2
4,1327,2015-06-02 11:12:29,1569,3
...,...,...,...,...
11459,310,2018-06-29 08:29:43,1569,11
11460,310,2018-06-29 11:34:08,1569,12
11461,310,2018-07-02 05:59:38,1569,12
11462,310,2018-07-02 07:25:23,1569,13


In [44]:
derates['active']

40          True
45          True
249        False
290         True
306        False
           ...  
1186072     True
1186195    False
1186207    False
1186730    False
1187331     True
Name: active, Length: 11495, dtype: bool

In [45]:
cumulative_active['spn'].unique()

array([1569, 5246])

In [46]:
faults_code=pd.read_excel('../data/Service Fault Codes_1_0_0_167.xlsx')

/Users/vahidehrasekhi/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [47]:
faults_code.head()

,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,SPN,J1939 FMI,J2012 Pcode,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
0,Y,111,167,Not Mapped,254,0,12,629,12,P0606,Red,Stop / Shutdown,Engine Control Module Critical Internal Failur...,Error internal to the ECM related to memory ha...
1,Y,112,167,Not Mapped,20,128,7,635,7,Not Mapped,Red,Stop / Shutdown,Engine Timing Actuator Driver Circuit - Mechan...,Mechanical failure in the engine timing actuat...
2,Y,113,167,Not Mapped,20,128,3,635,3,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,High signal voltage detected at the engine tim...
3,Y,114,167,Not Mapped,20,128,4,635,4,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,Low voltage detected at the engine timing actu...
4,Y,115,167,190,Not Mapped,Not Mapped,2,612,2,P0008,Red,Stop / Shutdown,Engine Magnetic Speed/Position Lost Both of Tw...,The ECM has detected that the primary and back...


In [48]:
faults_code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7124 entries, 0 to 7123
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Published in CES 14602  7124 non-null   object
 1   Cummins Fault Code      7124 non-null   int64 
 2   Revision                7124 non-null   int64 
 3   PID                     7124 non-null   object
 4   SID                     7124 non-null   object
 5   MID                     7124 non-null   object
 6   J1587 FMI               7124 non-null   int64 
 7   SPN                     7124 non-null   int64 
 8   J1939 FMI               7124 non-null   int64 
 9   J2012 Pcode             7124 non-null   object
 10  Lamp Color              7124 non-null   object
 11  Lamp Device             7124 non-null   object
 12  Cummins Description     7124 non-null   object
 13  Algorithm Description   2005 non-null   object
dtypes: int64(5), object(9)
memory usage: 779.3+ KB


In [49]:
faults_code['Cummins Description'].str.contains('Derate').value_counts()


False    7113
True       11
Name: Cummins Description, dtype: int64

In [54]:
derated_trucks = faults_code[(faults_code['SPN']==1569) | (faults_code['J1587 FMI']==31) |(faults_code['J1587 FMI']==31) | (faults_code['SPN']==5246)]

derated_trucks.head()

,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,SPN,J1939 FMI,J2012 Pcode,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
2518,Y,3712,167,Not Mapped,Not Mapped,Not Mapped,0,5246,0,Not Mapped,Red,Stop / Shutdown,Aftertreatment SCR Operator Inducement - Data ...,SCR inducement of 5 mph derate - Fault Code 41...
2520,Y,3714,167,Not Mapped,Not Mapped,Not Mapped,11,1569,31,Not Mapped,Amber,Warning,Engine Protection Torque Derate - Condition Ex...,NaN
2781,Y,4134,167,Not Mapped,Not Mapped,Not Mapped,0,5246,15,Not Mapped,Amber,Warning,Aftertreatment SCR Operator Inducement - Data ...,SCR inducement - Least Severe - Fault Code 371...
4338,Y,6254,167,Not Mapped,Not Mapped,Not Mapped,0,5246,16,Not Mapped,Amber,Warning,Aftertreatment SCR Operator Inducement Severit...,NaN
4339,Y,6255,167,Not Mapped,Not Mapped,Not Mapped,0,1569,15,Not Mapped,None,None,Engine Protection Torque Derate - Data Valid B...,NaN


In [52]:
faults_code['J1587 FMI'].unique()

array([12,  7,  3,  4,  2, 10,  0,  1,  5,  6, 11,  9, 13, 14,  8])

In [53]:
cummins_dec=faults_code[faults_code['Cummins Description'].str.contains('derate', case=False)]
cummins_dec

,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,SPN,J1939 FMI,J2012 Pcode,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
12,Y,124,167,102,Not Mapped,Not Mapped,0,102,16,P0234,Amber,Warning,Intake Manifold 1 Pressure - Data Valid But Ab...,High pressure detected in Intake Manifold #1.
13,Y,125,167,102,Not Mapped,Not Mapped,1,102,18,P0299,Amber,Warning,Intake Manifold 1 Pressure - Data Valid But Be...,Low pressure detected in Intake Manifold #1.
14,Y,126,167,Not Mapped,Not Mapped,Not Mapped,0,1129,16,Not Mapped,Amber,Warning,Intake Manifold 3 Pressure - Data Valid But Ab...,High pressure detected in Intake Manifold #2.
15,Y,127,167,Not Mapped,Not Mapped,Not Mapped,1,1129,18,Not Mapped,Amber,Warning,Intake Manifold 3 Pressure - Data Valid But Be...,Low pressure detected in Intake Manifold #2.
28,Y,142,167,100,Not Mapped,Not Mapped,0,100,16,Not Mapped,Amber,Warning,Engine Oil Rifle Pressure - Data Valid But Abo...,Engine oil pressure signal indicates engine oi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5737,Y,8191,167,Not Mapped,Not Mapped,Not Mapped,1,521213,18,Not Mapped,None,None,Turbocharger 3 Remaining Useful Life - Data Va...,NaN
5738,Y,8192,167,Not Mapped,Not Mapped,Not Mapped,1,521214,18,Not Mapped,Amber,Warning,Turbocharger 4 Remaining Useful Life - Data Va...,NaN
5739,Y,8193,167,Not Mapped,Not Mapped,Not Mapped,1,521214,18,Not Mapped,None,None,Turbocharger 4 Remaining Useful Life - Data Va...,NaN
5758,Y,8224,167,Not Mapped,Not Mapped,Not Mapped,0,521209,16,Not Mapped,None,None,Connectivity Module - Data Valid But Above Nor...,NaN
